<a href="https://colab.research.google.com/github/RomaricRichou/pie2021_option_swing/blob/main/SimulationsMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Exemple de référence

def american_mc(T,N,steps,S,r,sigma,K, typeCP):
  import numpy as np
	if typeCP=="call":
		typeCP=1
	else:
		typeCP=-1
	
	dt = np.float(T) / steps
	df = np.exp(-r * dt)
	# simulation of index levels
	SPath = np.zeros((steps + 1, N))
	SPath[0] = S
	
	z=np.random.randn(steps,N)

	for t in range(1, steps + 1):
		SPath[t] = SPath[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) *z[t-1])

	# case-based calculation of payoff
	h = np.maximum(typeCP*(SPath - K), 0)
	# LSM algorithm
	V = np.copy(h)
	for t in range(steps - 1, 0, -1):
		reg = np.polyfit(SPath[t], V[t + 1] * df, 7)
		C = np.polyval(reg, SPath[t])
		V[t] = np.where(C > h[t], V[t + 1] * df, h[t])
	# MCS estimator
	option = df * 1 / N * np.sum(V[1])
	return option

In [5]:
## Création d'un échantillon Normal : Df = F*sigma*dBt
def normal_mc(T,N,steps,S,r,sigma, show=False):
  import numpy as np
	
  dt = np.float(T) / steps
  # simulation of index levels
  SPath = np.zeros((steps + 1, N))
  SPath[0] = S
	
  z=np.random.randn(steps,N)

  for t in range(1, steps + 1):
    SPath[t] = SPath[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) *z[t-1])

  if show:
    import matplotlib.pyplot as plt
    time=[i*dt for i in range(steps+1)]
    for i in range(N):
      plt.plot(time, SPath[:,i])
    plt.show()
  return SPath

## Visualisation des simulations et détermination des quantiles (utiles en terme de gestion de risque ?) + Affichage temps de calcul
import numpy as np
import time

start=time.perf_counter()
mc_test=normal_mc(2,1000000,100,1,0,0.2,show = False)
end=time.perf_counter()


print("Les quantiles d'ordre 5%, 50%, 95%",np.quantile(mc_test[100],[0.05,0.5,0.95]))
print("Le pourcentile de 1 est :",np.percentile(mc_test[100],1))
print("Le temps de calcul est de :",end-start)

Les quantiles d'ordre 5%, 50%, 95% [0.60363567 0.96105357 1.5283488 ]
Le poucentile de 1 est : 0.49708146659925767
Le temps de calcul est de : 7.083642991000033
